# Index
1. Modeling
  1. E-R Diagrams
  2. Relational model
2. SQL
  1. SELECT
  2. JOIN
  3. Exercises
3. Using SQLAlchemy
4. Why do we normalize?
5. SQL security issues

# 1) Modeling

![a](http://actuallydata.co.uk/wordpress/wp-content/uploads/2015/02/databasecartoon-300x234.jpg)
When the amount or the complexity of the data we work with overwhelms us, we look for tools able to help us. Databases are one of the greatest tools for this job. There are several kinds of them, depending on the data they are designed to work with:

 - document database: books, blog posts, scientific papers, tweets, ...
 - timeseries database: weather station readings
 - graph database: friendships, likes, retweets
 - relational databases: customers, orders, providers, ...
 - and many more!

Even if in this course we will only discuss relational databases, it is important for you to know that other alternatives exists, and that they might be more suitable for some problem of yours in the future.

Relational databases are the ones mostly used because they are very versatile and can accommodate a very broad spectrum of data.

## 1.A) E-R Diagrams

![Data meaning](http://www.bigdata-madesimple.com/wp-content/uploads/2015/01/Accuracy-check.jpg)

To be able to store information in a relational database, it is essential to define the structure of the data, also called relational model. For this, we will use Entity-Relationship (E-R) diagrams.

An E-R diagram is composed from:
 - Entities: shall refer to common real-world concepts, and are describet by a set of attributes.
 - Relations: logical associations between entities, which may also have attributes. By cardinality, we find:
    - `1-1  -----`
    - `1-N  ----<`
    - `N-1  >----`
    - `N-N  >---<`

We will start with a basic example, defining the E-R diagram of the data needed to handle the enrollments of students in a university. As the starting point, we shall have a textual description of the problem:

*UAB offers a broad range of subjects for its students.
For each student, we must store their DNI, full name and birth date.
For each subject, we must store the name and its price.
Also, each subject is divided in several groups, so that we can adapt to the number and different schedule of the students. For each group, we must store its name, the schedule (either morning, afternoon or evening) and the teacher's name.
Finally, for each time a student enrolls in a subject, we must store a unique code, the date of the enrollment and whether they have paid yet or not.*

With this, we have enough information to build our E-R diagram. First, we shall identify the entities and its corresponding attributes:
 - student: dni, full name, birth date
 - subject: name, price
 - group: name, schedule, teacher name

We proceed identifying the relations:
 - A subject has several groups, but a group only corresponds to a unique subject
 - A student may be taking several subjects, enrolling on those groups which fit their schedule. In every group, there may be multiple students enrolled. Attributes: code, enrollment date, whether has been paid or not.

The resulting E-R diagram is a follows:

    Subject          Group                                  Student
    =======   ---<   ================   >---------------<   =======
     name             name               code                dni
     price            schedule           enrollment date     full name
                      teacher's name     has paid            birth date

## 1.B) Relational model

![ff](https://www.database-genius.com/cartoons/not_at_home/DBGenius_cartoon_not_at_home_screenres.png)

Next step is to transform that E-R diagram into a relational model specification. We shall adhere to the following convention:
 - Each entity becomes a table, mapping each attribute to a different column.
 - All values of a column belong to the same data domain (data type).
 - For each table, there shall be at least one subset of columns that uniquely identifies each row: it is called the primary key.
 - 1-1 / 1-N / N-1 relationships between tables are implemented by adding a subset of columns (at least the primary key) from the referenced table into the referencing one. These subsets are called foreign keys.
 - N-N relationships are implemented by unfolding them into a pair of 1-N / N-1 relationships with an intermediate table. This table will contain a foreign key to each of the referencing tables and any additional attribute defined for that relationship.

Among all the guidelines a good model should follow, the ones refering to "normalization" are a MUST. The science behind the normalization of a model is vast and complex, for now we can think of it as:
 - Values in a cell must be atomic
 - Values are stored in only one "place"
 - Columns not part of the PK, must depend ONLY on the PK

First, we define the tables:
 - Student: dni, full_name, birth_date
 - Subject: name, price
 - Group: name, schedule, teacher_name
 - Enrollment: code, enrollment_date, has_paid

Then, we define the primary keys (*) and foreign keys (~):
 - Student: *dni, full_name, birth_date
 - Subject: *name, price
 - Group: *name, schedule, teacher_name, ~subject_name
 - Enrollment: *code, date, has_paid, ~student_dni, ~group_name

And the diagram:

    Subject          Group                   Enrollment             Student
    =======   ---<   ==============   ---<   =============   >---   =======
    *name            *name                   *code                  *dni
     price            schedule                date                   full_name
                      teacher_name            has_paid               birth_date
                     ~subject_name           ~group_name
                                             ~student_dni

### Exercise

We will describe the data model of an online show shop:

*Our shop sells lots of different shoes. Each shoe has a brand, model and its price.
To be able to buy in our shop, customers must register their personal data, such as full name, email and password.
Customers may order multiple shoes, indicating the number of pairs for each shoe, the size and the color. For each order, we will store a unique code, the date and the shipping address.*

###  Solution (E-R)

In [ ]:
%load -r 1-16 solutions/12_01_Databases.py

### Solution (Relational model)

In [ ]:
%load -r 20-41 solutions/12_01_Databases.py

# 2) SQL

![a](https://comic.browserling.com/tables.png)

Once the relational model has been defined it needs to be implemented into the relational database.
Most interactions with relational databases are done through textual commands using a specific declarative language called SQL (Structured Query Language).

## 2.A) SELECT

SQL is very powerful, but for this course we will only have a look at the SELECT statement to filter, group, aggregate and retrieve information from the database.

Here is a simplified syntax of the SELECT statement:

    SELECT expression [, ...] 
    FROM table
    [ JOIN table ON condition ] 
    WHERE condition
    GROUP BY expression
    HAVING condition
    ORDER BY expression
    LIMIT number


To learn how to use the SELECT statement, we will use examples on top the relational models previously described.

### Select all columns from `student` table

    SELECT *
    FROM student

### Select the `name` of all `Subjects`, and retrieve only 3 entries

    SELECT name
    FROM subject
    LIMIT 3

### Select the average price of all `Subjects`

    SELECT AVG(price)
    FROM subject

### Select the name and price of the most expensive `Subject`

    SELECT name, price
    FROM subject
    ORDER BY price DESC
    LIMIT 1

### Select the names of all `Subjects` cheaper than 1000

    SELECT name
    FROM subject
    WHERE price < 1000

### Select how many `Students` we have

    SELECT COUNT(*)
    FROM student

### Select how many `Groups` we have, grouped by schedule

    SELECT schedule, COUNT(*)
    FROM group
    GROUP BY schedule

## 2.B) JOIN

We can also combine data from more than one table. Here is how we can use JOIN to achieve this.

### Select all the `Groups` with its corresponding `Subject`

    SELECT *
    FROM group
    JOIN subject
      ON group.subject_name = subject.name

### Select all `Subject` names given by a teacher named 'Ada Lovelace'

    SELECT subject.name
    FROM subject
    JOIN group
      ON subject.name = group.subject_name
    WHERE group.teacher = 'Ada Lovelace'

### Select the names of all `Students` who have some pending payment

    SELECT name
    FROM student
    JOIN enrollment
      ON student.id = enrollment.student_id
    GROUP BY student.id
    HAVING bool_and(group.has_paid) = FALSE

### Select the teachers of a `Student` with id 1234

    SELECT group.teacher
    FROM student
    JOIN enrollment
      ON student.id = enrollment.student_id
    JOIN group
      ON group.name = enrollment.group_name
    WHERE student.id = 1234

## 2.C) Exercises

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///../resources/shop.sqlite

### Select the brand, model and price of the cheapest shoe

In [ ]:
%load -r 50-59 solutions/12_01_Databases.py

### Count how many different models does each brand have

In [ ]:
%load -r 60-69 solutions/12_01_Databases.py

### Select how many pairs of shoes were sold in order number 4521

In [ ]:
%load -r 70-79 solutions/12_01_Databases.py

### How many shoes have been sold per brand?

In [ ]:
%load -r 80-89 solutions/12_01_Databases.py

### And per brand and model?

In [ ]:
%load -r 90-99 solutions/12_01_Databases.py

### Select the shipping address of the last 3 orders placed by the customer named 'Joan Clarke'

In [ ]:
%load -r 100-109 solutions/12_01_Databases.py

### Select the brand, model, color and size of all shows ever bought by the customer named 'Grace Hopper'

In [ ]:
%load -r 110-119 solutions/12_01_Databases.py

### How much did she spent?

In [ ]:
%load -r 130-139 solutions/12_01_Databases.py

# 3) Using SQLAlchemy

SQLAlchemy is a Python database toolkit that provides a unified and very powerful API for database access.

These are the steps to connect to a database and issue SQL statements:

### 1. Build a DSN (Data Source Name)

In [ ]:
dsn = "sqlite:///../resources/shop.sqlite"

### 2. Create an Engine and establish a connection

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(dsn)
conn = engine.connect()

### 3. Build the SQL statement

In [ ]:
from sqlalchemy.sql import text

sql = text(
    "SELECT * FROM shoe WHERE brand = :name"
).bindparams(
    name = 'Maggio'
)

### 4. Execute and retrieve results

In [ ]:
for shoe in conn.execute(sql):
    print("The «{name}» model from Maggio costs {price}".format(
        name = shoe.model,
        price = shoe.price
    ))

# 4) Why do we normalize?

Suppose you have the following table. Each row stores a galaxy id, its position and measured fluxes in several bands. Not all fluxes may be present, those missing will have a NULL value.

    Galaxy
    ===========
    *id
     ra
     dec
     flux_g
     flug_r
     flux_i
     flux_z
     flux_y

Before going on resolving the following SQL sentences, think about:
 - Is this model normalized?
 - What do we do if we want to measure on more bands?
 - How can we store also the flux_error for every band?

## Exercise

### Select id from galaxies with non-null flux on `g` band

In [ ]:
%load -r 150-159 solutions/12_01_Databases.py

### Select id from galaxies with all fluxes present

In [ ]:
%load -r 160-169 solutions/12_01_Databases.py

### Select all galaxies with 3 fluxes present

In [ ]:
%load -r 170-179 solutions/12_01_Databases.py

## Considerations

I cannot stress too much how important it is to have a good relational model in order to be able to work efficiently with all the data. 

Non-normalized models may appear "simpler" and "easier", but it is just a mirage. Behind the simple facade, such models are more difficult to maintain and evolve. Also, information present in them can be very hard to extract.

Not all data model requirements cannot be determined at the beginning. That means we must plan and prepare our data models for change. CHANGE IS UNAVOIDABLE, and data models must be able to adapt to the evolution of requirements.

## Exercise

Could you propose a normalized model?

### Solution

In [ ]:
%load -r 180-189 solutions/12_01_Databases.py

## Exercise

### Select id from galaxies with non-null flux on `g` band

In [ ]:
%load -r 190-199 solutions/12_01_Databases.py

### Select id from galaxies with 3 fluxes present

In [ ]:
%load -r 200-209 solutions/12_01_Databases.py

### Select id from galaxies with at least 3 fluxes present

In [ ]:
%load -r 210-219 solutions/12_01_Databases.py

## Considerations

 - What do we do if we want to measure on more bands?
 - How can we store also the flux_error for every band?
 - And the magnitude?
 - And the magnitude error?

# 5) SQL security issues

![SQL Injection](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)

Although mastering SQL is a must if we work with relational databases, it becomes tedious to manually write all those queries. Also, it is prone to errors and one has to validate each and every user-provided input or it could suffer from massive and fata security issues.

The most common security problem with hand-crafted SQL is SQL injection, where user provides some kind of parameter to the query. If this parameter is not secured enough or does not pass the proper validation, the user is efectibly able to run ANY statement on OUR database. It could steal our customers, our credentials, delete all our data, or worse, modifying critical data without our knowledge.

As an example, in our online shop we have a section to browse the shoes we sell. When a user select a particular brand, we display all the models and their price. Suppose we use the following query:

If we have a query like this one:

    SELECT brand, model, price
    FROM shoe
    WHERE brand = {$ parameter $}

### Normal use

    parameter = 'Maggio'

In [ ]:
%%sql    
    SELECT brand, model, price
    FROM shoe
    WHERE brand = 'Maggio'

### Issue 1: Ask for a non existing brand

    parameter = 'TheBestBrandInDaWorld'

In [ ]:
%%sql
    SELECT brand, model, price
    FROM shoe
    WHERE brand = 'TheBestBrandInDaWorld'

### Issue 2: Make the query fail

    parameter = ;

In [ ]:
%%sql
    SELECT brand, model, price
    FROM shoe
    WHERE brand = ;

### Issue 3: Select anything else

    parameter = '';SELECT * FROM customer

In [ ]:
%%sql
    SELECT brand, model, price
    FROM shoe
    WHERE brand = '';SELECT * FROM customer

### Issue 4: DO anything else

    parameter = '';DROP TABLE you_are_lucky_this_table_does_not_exist

In [ ]:
%%sql
    SELECT brand, model, price
    FROM shoe
    WHERE brand = '';DROP TABLE you_are_lucky_this_table_does_not_exist